In [1]:
import numpy as np
import pandas as pd
import re
import os

# DATASET 1

In [2]:
rating = pd.read_csv("./data/books_ratings_1.csv")
data = pd.read_csv("./data/books_1.csv")

### Q1: Título, autores y editoriales de los libros de categoría "Computers" entre 2000 y 2023 que contengan 'distributed' en su título.

In [3]:
year_regex = re.compile('[^\d]*(\d{4})[^\d]*')
def extract_year(x):
    if x:
        result = year_regex.search(x)
        return int(result.group(1)) if result else None
    return None

data_sanitized = data.dropna(subset=['Title', 'authors', 'categories', 'publishedDate'])
data_sanitized = data_sanitized[['Title', 'authors', 'publisher', 'categories', 'publishedDate']].copy()
data_sanitized['publishedYear'] = data_sanitized['publishedDate'].apply(extract_year)
data_sanitized['publishedDecade'] = data_sanitized['publishedYear'] // 10 * 10

data_filtered = data_sanitized.copy()
data_filtered = data_filtered[data_filtered['categories'] == "['Computers']"]
data_filtered = data_filtered[(data_filtered['publishedYear'] >= 2000) & (data_filtered['publishedYear'] <= 2023)]
data_filtered = data_filtered[data_filtered['Title'].str.contains('distributed', case=False)]
data_filtered[['Title', 'authors', 'publisher', 'publishedYear']]

,Title,authors,publisher,publishedYear
10998,Inside Distributed COM (Mps),"['Intermational Conference Ifip Tcs 200', 'Int...",Springer Science & Business Media,2000.0


### Q2: Autores que tengan títulos publicados en al menos 10 décadas distintas

In [4]:
## Autores que tengan títulos publicados en al menos 10 décadas distintas
authors_expanded = data_sanitized.explode('authors')
author_unique_decades = authors_expanded.groupby('authors')['publishedDecade'].nunique().reset_index(name='num_decades')
authors_with_at_least_ten_decades = author_unique_decades[author_unique_decades['num_decades'] >= 10]
authors_with_at_least_ten_decades

,authors,num_decades


### Q3: Títulos y autores de los libros publicados en la década del 90' con al menos 500 calificaciones

In [5]:
## Títulos publicados en la década del 90'
data_published_in_90s = data_sanitized[data_sanitized['publishedDecade'] == 1990]
print(f'Total 90 Data Titles: {data_published_in_90s.Title.count()}')
data_published_in_90s.head(3)

## Títulos publicados en la década del 90' con 500 calificaciones o más
rating_published_in_90s = rating.merge(data_published_in_90s, on='Title', how='inner')

# Normalizamos el titulo pasando a minusculas y removimendo espacios a izquierda y derecha
#obsolete: rating_published_date_90['Title'] = rating_published_date_90['Title'].str.lower().str.strip()

rating_published_in_90s_count_by_title = rating_published_in_90s.groupby('Title').size().reset_index(name="amountOfReviews")
rating_published_in_90s_count_by_title_more_than_500_reviews = rating_published_in_90s_count_by_title.loc[rating_published_in_90s_count_by_title['amountOfReviews'] >= 500]
rating_published_in_90s_count_by_title_more_than_500_reviews

# Join con tabla de libros para obtener autores
#obsolete: reduced_data = data[['Title', "authors"]].drop_duplicates()
reduced_data = data_sanitized[['Title', "authors"]]
#obsolete: reduced_data['Title'] = reduced_data['Title'].str.lower().str.strip()
rating_published_in_90s_count_by_title_more_than_500_reviews_with_authors = rating_published_in_90s_count_by_title_more_than_500_reviews.merge(reduced_data, on="Title", how="inner")
#obsolete: rating_published_date_90_count_by_title_more_than_500_reviews_with_authors.drop_duplicates()
rating_published_in_90s_count_by_title_more_than_500_reviews_with_authors

Total 90 Data Titles: 5328


,Title,amountOfReviews,authors
0,Adventures of Huckleberry Finn,1553,['Mark Twain']
1,Black Notice (Random House Large Print),654,['Patricia Daniels Cornwell']
2,Black and Blue: A Novel (Oprah's Book Club),500,['Anna Quindlen']
3,"Brave new world,: A novel",1053,['Aldous Huxley']
4,Foundation And Empire,609,['D. G. Leahy']
5,Foundation and Empire,1218,['D. G. Leahy']
6,Jane Eyre (New Windmill),1523,['Charlotte Brontë']
7,Little Women (Courage giant classics),836,['Louisa May Alcott']
8,OF MICE AND MEN,1348,['John Steinbeck']
9,One Hundred Years of Solitude,907,['Gabriel García Márquez']


### Q4: Top 10 libros con mejor promedio de calificación entre aquellos publicados en la década del 90’ con al menos 500 calificaciones

In [6]:
rating_date_90_at_least_500 = rating[['Title', 'review/score']].merge(
    rating_published_in_90s_count_by_title_more_than_500_reviews['Title'], on='Title', how='inner')
rating_date_90_at_least_500_mean_rating = rating_date_90_at_least_500.groupby(['Title']).mean().sort_values('review/score', ascending=False)
top_10_books = rating_date_90_at_least_500_mean_rating.head(10)
top_10_books

,review/score
Title,
The Killer Angels (Turtleback School & Library Binding Edition),4.663750
Jane Eyre (New Windmill),4.490479
Little Women (Courage giant classics),4.449761
The Grapes of Wrath,4.426166
The grapes of wrath,4.408560
OF MICE AND MEN,4.350148
"Slaughterhouse-five: Or, The children's crusade : a duty-dance with death",4.326653
Foundation and Empire,4.279146
Foundation And Empire,4.274220


### Q5: Título de libros de la categoría "Ficción" cuyo sentimiento de reseña promedio esté en el percentil 90 más alto

In [7]:
# Título de libros de la categoría "Ficción" cuyo sentimiento de reseña promedio sea esté en el percentil 90 más alto
from textblob import TextBlob
import ast

def is_fiction(s):
    v = ast.literal_eval(s)
    return 'fiction' in [c.lower() for c in v]

def analizar_sentimiento(texto):
    # TextBlob devuelve valores de polaridad de sentimiento en el rango de -1 a 1
    if isinstance(texto, str):
        blob = TextBlob(texto)
        sentimiento = blob.sentiment.polarity
        return sentimiento
    else:
        return None  # Si el texto no es una cadena válida, devolver None

# Sanitizamos el dataset para quedarnos con registros con categorías válidas
#obsolete:data_category_sanitized = data.dropna(subset=['categories'])

# Approach para quedarnos con registros relacionados a fiction
data_category_fiction = data_sanitized[data_sanitized['categories'].apply(is_fiction)]

# Generamos una versión reducida de reseñas que nos sirva para joinear
rating_reduced = rating[['Title', 'review/text']]

# Joineamos los libros de la categoría de Ficción con sus reseñas
data_category_fiction_merged_with_ratings = data_category_fiction.merge(rating_reduced, on='Title', how='inner')

# Calculamos el sentimiento sobre los registros de reseñas
data_category_fiction_merged_with_ratings['sentimentPolarity'] = data_category_fiction_merged_with_ratings['review/text'].apply(analizar_sentimiento)

# Agrupamos por título y calculamos el promedio de la sentimientPolarity

data_category_gruped_mean_sentiment_polarity = data_category_fiction_merged_with_ratings.groupby('Title')['sentimentPolarity'].mean().reset_index()

# Ordenamos los registros por polaridad de sentimiento en orden descendente
#obsolete: data_category_fiction_sorted = data_category_gruped_mean_sentiment_polarity.sort_values(by='sentimentPolarity', ascending=False)

# Calculamos el percentil 90
percentile_90 = data_category_gruped_mean_sentiment_polarity['sentimentPolarity'].quantile(0.9)

# Nos quedamos con los libros que esten en el percentil 90
data_category_fiction_percentile_90 = data_category_gruped_mean_sentiment_polarity[data_category_gruped_mean_sentiment_polarity['sentimentPolarity'] >= percentile_90]

data_category_fiction_percentile_90

,Title,sentimentPolarity
16,A Bad Girl's Money,0.341013
21,A Boss In A Million (Harlequin Presents No. 2095),0.406985
25,A Case of Loyalties,0.320833
27,A Christmas Carol (Enriched Classics (Pocket)),0.327589
40,A Deadly Quiet: An Inupiat Eskimo Mystery (Inu...,0.317500
...,...,...
4143,Wires & Watts,0.497778
4160,"Woman at the window, a novel",0.470833
4162,Women in the Wind,0.562500
4165,Words Of My Roaring,0.376465


# DATASET 2

In [8]:
rating = pd.read_csv("./data/books_ratings_2.csv")
data = pd.read_csv("./data/books_2.csv")

### Q1: Título, autores y editoriales de los libros de categoría "Computers" entre 2000 y 2023 que contengan 'distributed' en su título.

In [9]:
year_regex = re.compile('[^\d]*(\d{4})[^\d]*')
def extract_year(x):
    if x:
        result = year_regex.search(x)
        return int(result.group(1)) if result else None
    return None

data_sanitized = data.dropna(subset=['Title', 'authors', 'categories', 'publishedDate'])
data_sanitized = data_sanitized[['Title', 'authors', 'publisher', 'categories', 'publishedDate']].copy()
data_sanitized['publishedYear'] = data_sanitized['publishedDate'].apply(extract_year)
data_sanitized['publishedDecade'] = data_sanitized['publishedYear'] // 10 * 10

data_filtered = data_sanitized.copy()
data_filtered = data_filtered[data_filtered['categories'] == "['Computers']"]
data_filtered = data_filtered[(data_filtered['publishedYear'] >= 2000) & (data_filtered['publishedYear'] <= 2023)]
data_filtered = data_filtered[data_filtered['Title'].str.contains('distributed', case=False)]
data_filtered[['Title', 'authors', 'publisher', 'publishedYear']]

,Title,authors,publisher,publishedYear
4477,Distributed Com Application Development Using ...,['Jim Maloney'],Prentice Hall,2000.0
6864,Elements of Distributed Computing,['Vijay K. Garg'],John Wiley & Sons,2002.0
10281,ASP.NET Distributed Data Applications,"['Alex Homer', 'David Sussman']",Apress,2008.0
12202,Enterprise Linux at Work: How to Build 10 Dist...,['Stephen Asbury'],John Wiley & Sons Incorporated,2000.0


### Q2: Autores que tengan títulos publicados en al menos 10 décadas distintas

In [10]:
## Autores que tengan títulos publicados en al menos 10 décadas distintas
authors_expanded = data_sanitized.explode('authors')
author_unique_decades = authors_expanded.groupby('authors')['publishedDecade'].nunique().reset_index(name='num_decades')
authors_with_at_least_ten_decades = author_unique_decades[author_unique_decades['num_decades'] >= 10]
authors_with_at_least_ten_decades

,authors,num_decades


### Q3: Títulos y autores de los libros publicados en la década del 90' con al menos 500 calificaciones

In [11]:
## Títulos publicados en la década del 90'
data_published_in_90s = data_sanitized[data_sanitized['publishedDecade'] == 1990]
print(f'Total 90 Data Titles: {data_published_in_90s.Title.count()}')
data_published_in_90s.head(3)

## Títulos publicados en la década del 90' con 500 calificaciones o más
rating_published_in_90s = rating.merge(data_published_in_90s, on='Title', how='inner')

# Normalizamos el titulo pasando a minusculas y removimendo espacios a izquierda y derecha
#obsolete: rating_published_date_90['Title'] = rating_published_date_90['Title'].str.lower().str.strip()

rating_published_in_90s_count_by_title = rating_published_in_90s.groupby('Title').size().reset_index(name="amountOfReviews")
rating_published_in_90s_count_by_title_more_than_500_reviews = rating_published_in_90s_count_by_title.loc[rating_published_in_90s_count_by_title['amountOfReviews'] >= 500]
rating_published_in_90s_count_by_title_more_than_500_reviews

# Join con tabla de libros para obtener autores
#obsolete: reduced_data = data[['Title', "authors"]].drop_duplicates()
reduced_data = data_sanitized[['Title', "authors"]]
#obsolete: reduced_data['Title'] = reduced_data['Title'].str.lower().str.strip()
rating_published_in_90s_count_by_title_more_than_500_reviews_with_authors = rating_published_in_90s_count_by_title_more_than_500_reviews.merge(reduced_data, on="Title", how="inner")
#obsolete: rating_published_date_90_count_by_title_more_than_500_reviews_with_authors.drop_duplicates()
rating_published_in_90s_count_by_title_more_than_500_reviews_with_authors

Total 90 Data Titles: 5273


,Title,amountOfReviews,authors
0,A Christmas Carol [One Voice Recordings Edition],1002,['Charles Dickens']
1,ANIMAL FARM - A Fairy Story (Time Reading Prog...,1447,['Gillian Fenwick']
2,All Creatures Great and Small,824,['James Herriot']
3,Black Notice (Random House Large Print),654,['Patricia Daniels Cornwell']
4,Heart of Darkness (Everyman Classics),656,['Joseph Conrad']
5,I Am Legend,788,['Richard Matheson']
6,I know this much is true,1630,['Wally Lamb']
7,Little Women (Courage giant classics),836,['Louisa May Alcott']
8,Naked in Death,768,['J. D. Robb']
9,No man knows my history: The life of Joseph Sm...,636,['Fawn McKay Brodie']


### Q4: Top 10 libros con mejor promedio de calificación entre aquellos publicados en la década del 90’ con al menos 500 calificaciones

In [12]:
rating_date_90_at_least_500 = rating[['Title', 'review/score']].merge(
    rating_published_in_90s_count_by_title_more_than_500_reviews['Title'], on='Title', how='inner')
rating_date_90_at_least_500_mean_rating = rating_date_90_at_least_500.groupby(['Title']).mean().sort_values('review/score', ascending=False)
top_10_books = rating_date_90_at_least_500_mean_rating.head(10)
top_10_books

,review/score
Title,
All Creatures Great and Small,4.791262
A Christmas Carol [One Voice Recordings Edition],4.684631
The Killer Angels (Turtleback School & Library Binding Edition),4.663750
I know this much is true,4.500000
ANIMAL FARM - A Fairy Story (Time Reading Program Special Edition),4.458189
The hitchhiker's guide to the galaxy,4.457426
Little Women (Courage giant classics),4.449761
Naked in Death,4.381510
One Hundred Years of Solitude,4.239250


### Q5: Título de libros de la categoría "Ficción" cuyo sentimiento de reseña promedio esté en el percentil 90 más alto

In [13]:
# Título de libros de la categoría "Ficción" cuyo sentimiento de reseña promedio sea esté en el percentil 90 más alto
from textblob import TextBlob
import ast

def is_fiction(s):
    v = ast.literal_eval(s)
    return 'fiction' in [c.lower() for c in v]

def analizar_sentimiento(texto):
    # TextBlob devuelve valores de polaridad de sentimiento en el rango de -1 a 1
    if isinstance(texto, str):
        blob = TextBlob(texto)
        sentimiento = blob.sentiment.polarity
        return sentimiento
    else:
        return None  # Si el texto no es una cadena válida, devolver None

# Sanitizamos el dataset para quedarnos con registros con categorías válidas
#obsolete:data_category_sanitized = data.dropna(subset=['categories'])

# Approach para quedarnos con registros relacionados a fiction
data_category_fiction = data_sanitized[data_sanitized['categories'].apply(is_fiction)]

# Generamos una versión reducida de reseñas que nos sirva para joinear
rating_reduced = rating[['Title', 'review/text']]

# Joineamos los libros de la categoría de Ficción con sus reseñas
data_category_fiction_merged_with_ratings = data_category_fiction.merge(rating_reduced, on='Title', how='inner')

# Calculamos el sentimiento sobre los registros de reseñas
data_category_fiction_merged_with_ratings['sentimentPolarity'] = data_category_fiction_merged_with_ratings['review/text'].apply(analizar_sentimiento)

# Agrupamos por título y calculamos el promedio de la sentimientPolarity

data_category_gruped_mean_sentiment_polarity = data_category_fiction_merged_with_ratings.groupby('Title')['sentimentPolarity'].mean().reset_index()

# Ordenamos los registros por polaridad de sentimiento en orden descendente
#obsolete: data_category_fiction_sorted = data_category_gruped_mean_sentiment_polarity.sort_values(by='sentimentPolarity', ascending=False)

# Calculamos el percentil 90
percentile_90 = data_category_gruped_mean_sentiment_polarity['sentimentPolarity'].quantile(0.9)

# Nos quedamos con los libros que esten en el percentil 90
data_category_fiction_percentile_90 = data_category_gruped_mean_sentiment_polarity[data_category_gruped_mean_sentiment_polarity['sentimentPolarity'] >= percentile_90]

data_category_fiction_percentile_90

,Title,sentimentPolarity
1,'Tis The Season: The Choice\First Fruits\A New...,0.537500
5,2 Kill or Not to Kill: Two Christopher Raven N...,0.335146
7,"6 Chambers, 1 Bullet: A Novel",0.494167
9,"A Bad Day to Die: The Adventures of Lucius ""By...",0.322764
12,A Bite of Magic,0.330882
...,...,...
4151,Women I Have Known & Been,0.419231
4153,Women in the Wind,0.562500
4164,Wrangler (Hearts Of Wyoming) (Silhouette Speci...,0.417593
4182,Young Henry of Navarre,0.331330


# DATASET 3

In [14]:
rating = pd.read_csv("./data/books_ratings_3.csv")
data = pd.read_csv("./data/books_3.csv")

### Q1: Título, autores y editoriales de los libros de categoría "Computers" entre 2000 y 2023 que contengan 'distributed' en su título.

In [15]:
year_regex = re.compile('[^\d]*(\d{4})[^\d]*')
def extract_year(x):
    if x:
        result = year_regex.search(x)
        return int(result.group(1)) if result else None
    return None

data_sanitized = data.dropna(subset=['Title', 'authors', 'categories', 'publishedDate'])
data_sanitized = data_sanitized[['Title', 'authors', 'publisher', 'categories', 'publishedDate']].copy()
data_sanitized['publishedYear'] = data_sanitized['publishedDate'].apply(extract_year)
data_sanitized['publishedDecade'] = data_sanitized['publishedYear'] // 10 * 10

data_filtered = data_sanitized.copy()
data_filtered = data_filtered[data_filtered['categories'] == "['Computers']"]
data_filtered = data_filtered[(data_filtered['publishedYear'] >= 2000) & (data_filtered['publishedYear'] <= 2023)]
data_filtered = data_filtered[data_filtered['Title'].str.contains('distributed', case=False)]
data_filtered[['Title', 'authors', 'publisher', 'publishedYear']]

,Title,authors,publisher,publishedYear
16223,Malware: Fighting Malicious Code (Prentice Hal...,"['Ed Skoudis', 'Lenny Zeltser']",Prentice Hall Professional,2004.0


### Q2: Autores que tengan títulos publicados en al menos 10 décadas distintas

In [16]:
## Autores que tengan títulos publicados en al menos 10 décadas distintas
authors_expanded = data_sanitized.explode('authors')
author_unique_decades = authors_expanded.groupby('authors')['publishedDecade'].nunique().reset_index(name='num_decades')
authors_with_at_least_ten_decades = author_unique_decades[author_unique_decades['num_decades'] >= 10]
authors_with_at_least_ten_decades

,authors,num_decades


### Q3: Títulos y autores de los libros publicados en la década del 90' con al menos 500 calificaciones

In [17]:
## Títulos publicados en la década del 90'
data_published_in_90s = data_sanitized[data_sanitized['publishedDecade'] == 1990]
print(f'Total 90 Data Titles: {data_published_in_90s.Title.count()}')
data_published_in_90s.head(3)

## Títulos publicados en la década del 90' con 500 calificaciones o más
rating_published_in_90s = rating.merge(data_published_in_90s, on='Title', how='inner')

# Normalizamos el titulo pasando a minusculas y removimendo espacios a izquierda y derecha
#obsolete: rating_published_date_90['Title'] = rating_published_date_90['Title'].str.lower().str.strip()

rating_published_in_90s_count_by_title = rating_published_in_90s.groupby('Title').size().reset_index(name="amountOfReviews")
rating_published_in_90s_count_by_title_more_than_500_reviews = rating_published_in_90s_count_by_title.loc[rating_published_in_90s_count_by_title['amountOfReviews'] >= 500]
rating_published_in_90s_count_by_title_more_than_500_reviews

# Join con tabla de libros para obtener autores
#obsolete: reduced_data = data[['Title', "authors"]].drop_duplicates()
reduced_data = data_sanitized[['Title', "authors"]]
#obsolete: reduced_data['Title'] = reduced_data['Title'].str.lower().str.strip()
rating_published_in_90s_count_by_title_more_than_500_reviews_with_authors = rating_published_in_90s_count_by_title_more_than_500_reviews.merge(reduced_data, on="Title", how="inner")
#obsolete: rating_published_date_90_count_by_title_more_than_500_reviews_with_authors.drop_duplicates()
rating_published_in_90s_count_by_title_more_than_500_reviews_with_authors

Total 90 Data Titles: 5379


,Title,amountOfReviews,authors
0,A Portrait of the Artist As a Young Man,614,['James Joyce']
1,ANIMAL FARM - A Fairy Story (Time Reading Prog...,1447,['Gillian Fenwick']
2,Adventures of Huckleberry Finn,1553,['Mark Twain']
3,Black and Blue: A Novel (Oprah's Book Club),500,['Anna Quindlen']
4,"Brave new world,: A novel",1053,['Aldous Huxley']
5,Catch 22,2084,['Joseph Heller']
6,Heart of Darkness,788,['Joseph Conrad']
7,I Am Legend,788,['Richard Matheson']
8,Jane Eyre (New Windmill),1523,['Charlotte Brontë']
9,No man knows my history: The life of Joseph Sm...,636,['Fawn McKay Brodie']


### Q4: Top 10 libros con mejor promedio de calificación entre aquellos publicados en la década del 90’ con al menos 500 calificaciones

In [18]:
rating_date_90_at_least_500 = rating[['Title', 'review/score']].merge(
    rating_published_in_90s_count_by_title_more_than_500_reviews['Title'], on='Title', how='inner')
rating_date_90_at_least_500_mean_rating = rating_date_90_at_least_500.groupby(['Title']).mean().sort_values('review/score', ascending=False)
top_10_books = rating_date_90_at_least_500_mean_rating.head(10)
top_10_books

,review/score
Title,
The Killer Angels (Turtleback School & Library Binding Edition),4.663750
The Secret Garden,4.539911
Jane Eyre (New Windmill),4.490479
ANIMAL FARM - A Fairy Story (Time Reading Program Special Edition),4.458189
The Grapes of Wrath (Steinbeck Essentials),4.407022
Catch 22,4.391075
The Electric Kool-Aid Acid Test,4.324100
Slaughter-House Five,4.324000
"Brave new world,: A novel",4.235518


### Q5: Título de libros de la categoría "Ficción" cuyo sentimiento de reseña promedio esté en el percentil 90 más alto

In [19]:
# Título de libros de la categoría "Ficción" cuyo sentimiento de reseña promedio sea esté en el percentil 90 más alto
from textblob import TextBlob
import ast

def is_fiction(s):
    v = ast.literal_eval(s)
    return 'fiction' in [c.lower() for c in v]

def analizar_sentimiento(texto):
    # TextBlob devuelve valores de polaridad de sentimiento en el rango de -1 a 1
    if isinstance(texto, str):
        blob = TextBlob(texto)
        sentimiento = blob.sentiment.polarity
        return sentimiento
    else:
        return None  # Si el texto no es una cadena válida, devolver None

# Sanitizamos el dataset para quedarnos con registros con categorías válidas
#obsolete:data_category_sanitized = data.dropna(subset=['categories'])

# Approach para quedarnos con registros relacionados a fiction
data_category_fiction = data_sanitized[data_sanitized['categories'].apply(is_fiction)]

# Generamos una versión reducida de reseñas que nos sirva para joinear
rating_reduced = rating[['Title', 'review/text']]

# Joineamos los libros de la categoría de Ficción con sus reseñas
data_category_fiction_merged_with_ratings = data_category_fiction.merge(rating_reduced, on='Title', how='inner')

# Calculamos el sentimiento sobre los registros de reseñas
data_category_fiction_merged_with_ratings['sentimentPolarity'] = data_category_fiction_merged_with_ratings['review/text'].apply(analizar_sentimiento)

# Agrupamos por título y calculamos el promedio de la sentimientPolarity

data_category_gruped_mean_sentiment_polarity = data_category_fiction_merged_with_ratings.groupby('Title')['sentimentPolarity'].mean().reset_index()

# Ordenamos los registros por polaridad de sentimiento en orden descendente
#obsolete: data_category_fiction_sorted = data_category_gruped_mean_sentiment_polarity.sort_values(by='sentimentPolarity', ascending=False)

# Calculamos el percentil 90
percentile_90 = data_category_gruped_mean_sentiment_polarity['sentimentPolarity'].quantile(0.9)

# Nos quedamos con los libros que esten en el percentil 90
data_category_fiction_percentile_90 = data_category_gruped_mean_sentiment_polarity[data_category_gruped_mean_sentiment_polarity['sentimentPolarity'] >= percentile_90]

data_category_fiction_percentile_90

,Title,sentimentPolarity
1,"""Ra Force Rising"": Brother G (Shades of Memnon)",0.334209
5,100 Great Fantasy Short Short Stories,0.408667
6,"100: Sudden Fiction, Sexual Fiction",0.432470
11,"20th Century American Short Stories, Revised E...",0.540000
21,A Caretaker of Love,0.333333
...,...,...
4104,Wizard at Large Book Three The Magic Kingdom o...,0.612500
4105,Wolf :Chinese,0.325000
4148,Zoboa,0.533333
4149,Zora Neale Hurston : Novels and Stories : Jona...,0.338790


# DATASET 4

In [20]:
rating = pd.read_csv("./data/books_ratings_4.csv")
data = pd.read_csv("./data/books_4.csv")

### Q1: Título, autores y editoriales de los libros de categoría "Computers" entre 2000 y 2023 que contengan 'distributed' en su título.

In [21]:
year_regex = re.compile('[^\d]*(\d{4})[^\d]*')
def extract_year(x):
    if x:
        result = year_regex.search(x)
        return int(result.group(1)) if result else None
    return None

data_sanitized = data.dropna(subset=['Title', 'authors', 'categories', 'publishedDate'])
data_sanitized = data_sanitized[['Title', 'authors', 'publisher', 'categories', 'publishedDate']].copy()
data_sanitized['publishedYear'] = data_sanitized['publishedDate'].apply(extract_year)
data_sanitized['publishedDecade'] = data_sanitized['publishedYear'] // 10 * 10

data_filtered = data_sanitized.copy()
data_filtered = data_filtered[data_filtered['categories'] == "['Computers']"]
data_filtered = data_filtered[(data_filtered['publishedYear'] >= 2000) & (data_filtered['publishedYear'] <= 2023)]
data_filtered = data_filtered[data_filtered['Title'].str.contains('distributed', case=False)]
data_filtered[['Title', 'authors', 'publisher', 'publishedYear']]

,Title,authors,publisher,publishedYear
18091,IBM&reg; WebSphere&reg; Application Server for...,['James W. Cortada'],MIT Press,2019.0
19488,Distributed Systems Architecture: A Middleware...,"['Arno Puder', 'Kay Römer', 'Frank Pilhofer']",Elsevier,2011.0
24519,Elements of Distributed Computing,['Vijay K. Garg'],John Wiley & Sons,2002.0


### Q2: Autores que tengan títulos publicados en al menos 10 décadas distintas

In [22]:
## Autores que tengan títulos publicados en al menos 10 décadas distintas
authors_expanded = data_sanitized.explode('authors')
author_unique_decades = authors_expanded.groupby('authors')['publishedDecade'].nunique().reset_index(name='num_decades')
authors_with_at_least_ten_decades = author_unique_decades[author_unique_decades['num_decades'] >= 10]
authors_with_at_least_ten_decades

,authors,num_decades


### Q3: Títulos y autores de los libros publicados en la década del 90' con al menos 500 calificaciones

In [23]:
## Títulos publicados en la década del 90'
data_published_in_90s = data_sanitized[data_sanitized['publishedDecade'] == 1990]
print(f'Total 90 Data Titles: {data_published_in_90s.Title.count()}')
data_published_in_90s.head(3)

## Títulos publicados en la década del 90' con 500 calificaciones o más
rating_published_in_90s = rating.merge(data_published_in_90s, on='Title', how='inner')

# Normalizamos el titulo pasando a minusculas y removimendo espacios a izquierda y derecha
#obsolete: rating_published_date_90['Title'] = rating_published_date_90['Title'].str.lower().str.strip()

rating_published_in_90s_count_by_title = rating_published_in_90s.groupby('Title').size().reset_index(name="amountOfReviews")
rating_published_in_90s_count_by_title_more_than_500_reviews = rating_published_in_90s_count_by_title.loc[rating_published_in_90s_count_by_title['amountOfReviews'] >= 500]
rating_published_in_90s_count_by_title_more_than_500_reviews

# Join con tabla de libros para obtener autores
#obsolete: reduced_data = data[['Title', "authors"]].drop_duplicates()
reduced_data = data_sanitized[['Title', "authors"]]
#obsolete: reduced_data['Title'] = reduced_data['Title'].str.lower().str.strip()
rating_published_in_90s_count_by_title_more_than_500_reviews_with_authors = rating_published_in_90s_count_by_title_more_than_500_reviews.merge(reduced_data, on="Title", how="inner")
#obsolete: rating_published_date_90_count_by_title_more_than_500_reviews_with_authors.drop_duplicates()
rating_published_in_90s_count_by_title_more_than_500_reviews_with_authors

Total 90 Data Titles: 5320


,Title,amountOfReviews,authors
0,A Tree Grows in Brooklyn,3904,['Betty Smith']
1,"A Tree Grows in Brooklyn by Smith, Betty",786,['Betty Smith']
2,ANIMAL FARM - A Fairy Story (Time Reading Prog...,1447,['Gillian Fenwick']
3,Catch 22,2084,['Joseph Heller']
4,Catch 22 (catch-22),1044,['Joseph Heller']
5,I know this much is true,1630,['Wally Lamb']
6,Jane Eyre (Everyman's Classics),1523,['Charlotte Brontë']
7,Killing Time,538,['Paul Feyerabend']
8,The Adventures of Tom Sawyer (Courage Literary...,674,['Mark Twain']
9,The Grapes of Wrath,772,['John Steinbeck']


### Q4: Top 10 libros con mejor promedio de calificación entre aquellos publicados en la década del 90’ con al menos 500 calificaciones

In [24]:
rating_date_90_at_least_500 = rating[['Title', 'review/score']].merge(
    rating_published_in_90s_count_by_title_more_than_500_reviews['Title'], on='Title', how='inner')
rating_date_90_at_least_500_mean_rating = rating_date_90_at_least_500.groupby(['Title']).mean().sort_values('review/score', ascending=False)
top_10_books = rating_date_90_at_least_500_mean_rating.head(10)
top_10_books

,review/score
Title,
"A Tree Grows in Brooklyn by Smith, Betty",4.707379
A Tree Grows in Brooklyn,4.706711
The Secret Garden,4.539911
I know this much is true,4.500000
Jane Eyre (Everyman's Classics),4.490479
ANIMAL FARM - A Fairy Story (Time Reading Program Special Edition),4.458189
The Grapes of Wrath,4.426166
Catch 22 (catch-22),4.391762
Catch 22,4.391075


### Q5: Título de libros de la categoría "Ficción" cuyo sentimiento de reseña promedio esté en el percentil 90 más alto

In [25]:
# Título de libros de la categoría "Ficción" cuyo sentimiento de reseña promedio sea esté en el percentil 90 más alto
from textblob import TextBlob
import ast

def is_fiction(s):
    v = ast.literal_eval(s)
    return 'fiction' in [c.lower() for c in v]

def analizar_sentimiento(texto):
    # TextBlob devuelve valores de polaridad de sentimiento en el rango de -1 a 1
    if isinstance(texto, str):
        blob = TextBlob(texto)
        sentimiento = blob.sentiment.polarity
        return sentimiento
    else:
        return None  # Si el texto no es una cadena válida, devolver None

# Sanitizamos el dataset para quedarnos con registros con categorías válidas
#obsolete:data_category_sanitized = data.dropna(subset=['categories'])

# Approach para quedarnos con registros relacionados a fiction
data_category_fiction = data_sanitized[data_sanitized['categories'].apply(is_fiction)]

# Generamos una versión reducida de reseñas que nos sirva para joinear
rating_reduced = rating[['Title', 'review/text']]

# Joineamos los libros de la categoría de Ficción con sus reseñas
data_category_fiction_merged_with_ratings = data_category_fiction.merge(rating_reduced, on='Title', how='inner')

# Calculamos el sentimiento sobre los registros de reseñas
data_category_fiction_merged_with_ratings['sentimentPolarity'] = data_category_fiction_merged_with_ratings['review/text'].apply(analizar_sentimiento)

# Agrupamos por título y calculamos el promedio de la sentimientPolarity

data_category_gruped_mean_sentiment_polarity = data_category_fiction_merged_with_ratings.groupby('Title')['sentimentPolarity'].mean().reset_index()

# Ordenamos los registros por polaridad de sentimiento en orden descendente
#obsolete: data_category_fiction_sorted = data_category_gruped_mean_sentiment_polarity.sort_values(by='sentimentPolarity', ascending=False)

# Calculamos el percentil 90
percentile_90 = data_category_gruped_mean_sentiment_polarity['sentimentPolarity'].quantile(0.9)

# Nos quedamos con los libros que esten en el percentil 90
data_category_fiction_percentile_90 = data_category_gruped_mean_sentiment_polarity[data_category_gruped_mean_sentiment_polarity['sentimentPolarity'] >= percentile_90]

data_category_fiction_percentile_90

,Title,sentimentPolarity
0,"""Ra Force Rising"": Brother G (Shades of Memnon)",0.334209
15,666: a novel,0.583333
20,A Bite of Magic,0.330882
43,A Deadly Quiet: An Inupiat Eskimo Mystery (Inu...,0.317500
48,A Family Affair (Zebra Regency Romance),0.436818
...,...,...
4142,Wings of Healing,0.447798
4143,"Wings of Morning (These Highland Hills, Book 2)",0.375898
4158,Wolf :Chinese,0.325000
4160,Wolf Mountain (Plainsmen),0.448047


# DATASET 5

In [26]:
rating = pd.read_csv("./data/books_ratings_5.csv")
data = pd.read_csv("./data/books_5.csv")

### Q1: Título, autores y editoriales de los libros de categoría "Computers" entre 2000 y 2023 que contengan 'distributed' en su título.

In [27]:
year_regex = re.compile('[^\d]*(\d{4})[^\d]*')
def extract_year(x):
    if x:
        result = year_regex.search(x)
        return int(result.group(1)) if result else None
    return None

data_sanitized = data.dropna(subset=['Title', 'authors', 'categories', 'publishedDate'])
data_sanitized = data_sanitized[['Title', 'authors', 'publisher', 'categories', 'publishedDate']].copy()
data_sanitized['publishedYear'] = data_sanitized['publishedDate'].apply(extract_year)
data_sanitized['publishedDecade'] = data_sanitized['publishedYear'] // 10 * 10

data_filtered = data_sanitized.copy()
data_filtered = data_filtered[data_filtered['categories'] == "['Computers']"]
data_filtered = data_filtered[(data_filtered['publishedYear'] >= 2000) & (data_filtered['publishedYear'] <= 2023)]
data_filtered = data_filtered[data_filtered['Title'].str.contains('distributed', case=False)]
data_filtered[['Title', 'authors', 'publisher', 'publishedYear']]

,Title,authors,publisher,publishedYear
19189,Inside Distributed COM (Mps),"['Intermational Conference Ifip Tcs 200', 'Int...",Springer Science & Business Media,2000.0


### Q2: Autores que tengan títulos publicados en al menos 10 décadas distintas

In [28]:
## Autores que tengan títulos publicados en al menos 10 décadas distintas
authors_expanded = data_sanitized.explode('authors')
author_unique_decades = authors_expanded.groupby('authors')['publishedDecade'].nunique().reset_index(name='num_decades')
authors_with_at_least_ten_decades = author_unique_decades[author_unique_decades['num_decades'] >= 10]
authors_with_at_least_ten_decades

,authors,num_decades


### Q3: Títulos y autores de los libros publicados en la década del 90' con al menos 500 calificaciones

In [29]:
## Títulos publicados en la década del 90'
data_published_in_90s = data_sanitized[data_sanitized['publishedDecade'] == 1990]
print(f'Total 90 Data Titles: {data_published_in_90s.Title.count()}')
data_published_in_90s.head(3)

## Títulos publicados en la década del 90' con 500 calificaciones o más
rating_published_in_90s = rating.merge(data_published_in_90s, on='Title', how='inner')

# Normalizamos el titulo pasando a minusculas y removimendo espacios a izquierda y derecha
#obsolete: rating_published_date_90['Title'] = rating_published_date_90['Title'].str.lower().str.strip()

rating_published_in_90s_count_by_title = rating_published_in_90s.groupby('Title').size().reset_index(name="amountOfReviews")
rating_published_in_90s_count_by_title_more_than_500_reviews = rating_published_in_90s_count_by_title.loc[rating_published_in_90s_count_by_title['amountOfReviews'] >= 500]
rating_published_in_90s_count_by_title_more_than_500_reviews

# Join con tabla de libros para obtener autores
#obsolete: reduced_data = data[['Title', "authors"]].drop_duplicates()
reduced_data = data_sanitized[['Title', "authors"]]
#obsolete: reduced_data['Title'] = reduced_data['Title'].str.lower().str.strip()
rating_published_in_90s_count_by_title_more_than_500_reviews_with_authors = rating_published_in_90s_count_by_title_more_than_500_reviews.merge(reduced_data, on="Title", how="inner")
#obsolete: rating_published_date_90_count_by_title_more_than_500_reviews_with_authors.drop_duplicates()
rating_published_in_90s_count_by_title_more_than_500_reviews_with_authors

Total 90 Data Titles: 5452


,Title,amountOfReviews,authors
0,A Farewell to Arms,1475,['Ernest Hemingway']
1,ANIMAL FARM - A Fairy Story (Time Reading Prog...,1447,['Gillian Fenwick']
2,Black Notice (Random House Large Print),654,['Patricia Daniels Cornwell']
3,"Brave new world,: A novel",1053,['Aldous Huxley']
4,Little Women (Courage giant classics),836,['Louisa May Alcott']
5,Point of origin,521,['Patricia Daniels Cornwell']
6,Red Tent,1736,['Anita Diamant']
7,The Good Earth,602,"['Pearl S. Buck', 'Emily Hutchinson']"
8,The Grapes of Wrath,772,['John Steinbeck']
9,The Killer Angels (Turtleback School & Library...,800,['Michael Shaara']


### Q4: Top 10 libros con mejor promedio de calificación entre aquellos publicados en la década del 90’ con al menos 500 calificaciones

In [30]:
rating_date_90_at_least_500 = rating[['Title', 'review/score']].merge(
    rating_published_in_90s_count_by_title_more_than_500_reviews['Title'], on='Title', how='inner')
rating_date_90_at_least_500_mean_rating = rating_date_90_at_least_500.groupby(['Title']).mean().sort_values('review/score', ascending=False)
top_10_books = rating_date_90_at_least_500_mean_rating.head(10)
top_10_books

,review/score
Title,
The Killer Angels (Turtleback School & Library Binding Edition),4.663750
ANIMAL FARM - A Fairy Story (Time Reading Program Special Edition),4.458189
Little Women (Courage giant classics),4.449761
The Grapes of Wrath,4.426166
"Brave new world,: A novel",4.235518
The Good Earth,4.204319
The Red Tent (Bestselling Backlist),4.153451
Red Tent,4.151498
A Farewell to Arms,3.955254


### Q5: Título de libros de la categoría "Ficción" cuyo sentimiento de reseña promedio esté en el percentil 90 más alto

In [31]:
# Título de libros de la categoría "Ficción" cuyo sentimiento de reseña promedio sea esté en el percentil 90 más alto
from textblob import TextBlob
import ast

def is_fiction(s):
    v = ast.literal_eval(s)
    return 'fiction' in [c.lower() for c in v]

def analizar_sentimiento(texto):
    # TextBlob devuelve valores de polaridad de sentimiento en el rango de -1 a 1
    if isinstance(texto, str):
        blob = TextBlob(texto)
        sentimiento = blob.sentiment.polarity
        return sentimiento
    else:
        return None  # Si el texto no es una cadena válida, devolver None

# Sanitizamos el dataset para quedarnos con registros con categorías válidas
#obsolete:data_category_sanitized = data.dropna(subset=['categories'])

# Approach para quedarnos con registros relacionados a fiction
data_category_fiction = data_sanitized[data_sanitized['categories'].apply(is_fiction)]

# Generamos una versión reducida de reseñas que nos sirva para joinear
rating_reduced = rating[['Title', 'review/text']]

# Joineamos los libros de la categoría de Ficción con sus reseñas
data_category_fiction_merged_with_ratings = data_category_fiction.merge(rating_reduced, on='Title', how='inner')

# Calculamos el sentimiento sobre los registros de reseñas
data_category_fiction_merged_with_ratings['sentimentPolarity'] = data_category_fiction_merged_with_ratings['review/text'].apply(analizar_sentimiento)

# Agrupamos por título y calculamos el promedio de la sentimientPolarity

data_category_gruped_mean_sentiment_polarity = data_category_fiction_merged_with_ratings.groupby('Title')['sentimentPolarity'].mean().reset_index()

# Ordenamos los registros por polaridad de sentimiento en orden descendente
#obsolete: data_category_fiction_sorted = data_category_gruped_mean_sentiment_polarity.sort_values(by='sentimentPolarity', ascending=False)

# Calculamos el percentil 90
percentile_90 = data_category_gruped_mean_sentiment_polarity['sentimentPolarity'].quantile(0.9)

# Nos quedamos con los libros que esten en el percentil 90
data_category_fiction_percentile_90 = data_category_gruped_mean_sentiment_polarity[data_category_gruped_mean_sentiment_polarity['sentimentPolarity'] >= percentile_90]

data_category_fiction_percentile_90

,Title,sentimentPolarity
3,"100: Sudden Fiction, Sexual Fiction",0.432470
13,"6 Chambers, 1 Bullet: A Novel",0.494167
15,A Bad Girl's Money,0.341013
32,A Christmas Carol (Enriched Classics (Pocket)),0.327589
33,A Christmas Kiss and Winter Wonderland (Signet...,0.340052
...,...,...
4121,William Faulkner,0.388810
4128,Winning his way,0.349026
4143,Wolf Mountain (Plainsmen),0.448047
4151,Women Before the Bench,0.329924
